## Day 18: Boiling Boulders

You and the elephants finally reach fresh air. You've emerged near the base of a large volcano that seems to be actively erupting! Fortunately, the lava seems to be flowing away from you and toward the ocean.

Bits of lava are still being ejected toward you, so you're sheltering in the cavern exit a little longer. Outside the cave, you can see the lava landing in a pond and hear it loudly hissing as it solidifies.

Depending on the specific compounds in the lava and speed at which it cools, it might be forming obsidian! The cooling rate should be based on the surface area of the lava droplets, so you take a quick scan of a droplet as it flies past you (your puzzle input).

Because of how quickly the lava is moving, the scan isn't very good; its resolution is quite low and, as a result, it approximates the shape of the lava droplet with **1x1x1 cubes on a 3D grid**, each given as its x,y,z position.

To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like 1,1,1 and 2,1,1, each cube would have a single side covered and five sides exposed, a total surface area of 10 sides.

Here's a larger example:

```
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
```

In [18]:
import collections
import itertools

import pytest
from icecream import ic

In [19]:
def load_from_file(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            x, y, z = line.split(',')
            yield int(x), int(y), int(z)

In [20]:
def load_input(filename):
    space_map = collections.defaultdict(bool)
    blocks = []
    min_x = min_y = min_z = float("inf")
    max_x = max_y = max_z = float("-inf")
    for x, y, z in load_from_file(filename):
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        min_z = min(min_z, z)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
        max_z = max(max_z, z)
        space_map[(x, y, z)] = True
        blocks.append((x, y, z))
    bounds = (
        (min_x, max_x + 1),
        (min_y, max_y + 1),
        (min_z, max_z + 1),
    )
    return (blocks, space_map, bounds)

> Una función para contar las caras libres:

In [21]:
def count_free_sides(data, coord):
    x, y, z = coord
    return 6 - sum(
        [
            data[(x - 1, y, z)],
            data[(x + 1, y, z)],
            data[(x, y - 1, z)],
            data[(x, y + 1, z)],
            data[(x, y, z - 1)],
            data[(x, y, z + 1)],
        ]
    )


test_data = collections.defaultdict(bool)
test_data[(1, 1, 1)] = True
test_data[(2, 1, 1)] = True
assert count_free_sides(test_data, (1, 1, 1)) == 5
assert count_free_sides(test_data, (2, 1, 1)) == 5

In [22]:
_blocks, cube_map, _bounds = load_input("cube.txt")
assert len(cube_map) == 26
assert cube_map[1, 3, -1] is True
assert count_free_sides(cube_map, (1, 3, -1)) == 3
assert cube_map[1, 3, 0] is True
assert count_free_sides(cube_map, (1, 3, 0)) == 2
assert cube_map[1, 3, 1] is True
assert count_free_sides(cube_map, (1, 3, 1)) == 3

assert cube_map[1, 2, 0] is True
assert count_free_sides(cube_map, [1, 2, 0]) == 2

assert cube_map[2, 2, 0] is False
assert count_free_sides(cube_map, [2, 2, 0]) == 0

assert cube_map[2, 2, -1] is True
assert count_free_sides(cube_map, [2, 2, -1]) == 2

assert solution_one("cube.txt") == 60

In the above example, after counting up all the sides that aren't connected to another cube, the total surface area is **64**.

**What is the surface area of your scanned lava droplet?**

In [23]:
def solution_one(filename):
    blocks, space_map, _ = load_input(filename)
    acc = 0
    for coord in blocks:
        acc += count_free_sides(space_map, coord)
    return acc

In [24]:
assert solution_one('sample.txt') == 64

In [25]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")

Solution part one: 4548


## Part two

Something seems off about your calculation. The cooling rate depends on exterior surface area, but your calculation also included the surface area of air pockets trapped in the lava droplet.

Instead, consider only cube sides that could be reached by the water and steam as the lava droplet tumbles into the pond. The steam will expand to reach as much as possible, completely displacing any air on the outside of the lava droplet but never expanding diagonally.

In the larger example above, exactly one cube of air is trapped within the lava droplet (at 2,2,5), so the exterior surface area of the lava droplet is **58**.

> Aquí el problema es saber si una celda esta dentro o no. Para cada celda libre que está dentro de los límites del _bounding box_, hay que ver si se puede trazar una ruta hasta el exxterior.

In [26]:
def is_empty(_map, coord):
    return _map[coord] is False


def z_section(_map, bounds, z):
    (min_x, max_x), (min_y, max_y), (min_z, max_z) = bounds
    assert z in range(min_z, max_z)
    buff = [f'Z : {z}\n']
    for y in range(max_y, min(min_y, 0)-1, -1):
        for x in range(min(min_x, 0), max_x+1):
            if is_empty(_map, (x, y, z)):
                buff.append('.')
            else:
                buff.append('#')
        buff.append('\n')
    return ''.join(buff)


def test_cube():
    blocks, cube_map, bounds = load_input("cube.txt")
    for z in range(-1, 2):
        print(z_section(cube_map, bounds, z))
    for x, y, z in itertools.product(range(1, 4), range(1, 4), range(-1, 2)):
        if (x, y, z) == (2, 2, 0):
            assert is_empty(cube_map, (x, y, z))
        else:
            assert not is_empty(cube_map, (x, y, z))


def test_z_section():
    blocks, box_map, bounds = load_input("box.txt")
    for z in range(-1, 2):
        print(z_section(box_map, bounds, z))


def get_all_neighbours(_map, coord):
    (x, y, z) = coord
    deltas = [
        (-1, 0, 0),
        (1, 0, 0),
        (0, -1, 0),
        (0, 1, 0),
        (0, 0, -1),
        (0, 0, 1),
        ]
    for (dx, dy, dz) in deltas:
        yield (x+dx, y+dy, z+dz)
    return


def get_empty_neighbours(_map, coord):
    yield from [
        node
        for node in get_all_neighbours(_map, coord)
        if is_empty(_map, node)
    ]

def get_local_neighbours(_map, coord):
    yield from [
        node
        for node in get_all_neighbours(_map, coord)
        if not is_empty(_map, node)
    ]


def test_get_neighbours():
    blocks, _map, bounds = load_input("cube.txt")
    assert set(get_empty_neighbours(_map, (1, 3, -1))) == set([
        (0, 3, -1),
        (1, 4, -1),
        (1, 3, -2),
        ])
    assert set(get_local_neighbours(_map, (1, 3, -1))) == set([
        (2, 3, -1),
        (1, 3, 0),
        (1, 2, -1),
        ])
    assert set(get_empty_neighbours(_map, (1, 3, -1))) == set([
        (0, 3, -1),
        (1, 4, -1),
        (1, 3, -2),
        ])
    assert set(get_all_neighbours(_map, (1, 3, -1))) == set([
        (0, 3, -1),
        (1, 2, -1),
        (1, 3, -2),
        (1, 3, 0),
        (1, 4, -1),
        (2, 3, -1),
        ])


test_get_neighbours()

In [27]:
def is_inside(coord, bounds):
    (x, y, z) = coord
    (min_x, max_x), (min_y, max_y), (min_z, max_z) = bounds
    return all([
        min_x <= x < max_x,
        min_y <= y < max_y,
        min_z <= z < max_z,
        ])


def find_exit(_map, coord, bounds, tron=False):
    if tron:
        ic('find_exit starts')
    if not is_inside(coord, bounds):
        if tron:
            ic(f'{coord} It is outside!')
        return True
    (min_x, max_x), (min_y, max_y), (min_z, max_z) = bounds
    frontier = set([coord])
    visited = set()
    while frontier:
        if tron:
            ic(visited, frontier)
        node = frontier.pop()
        if not is_inside(node, bounds):
            if tron:
                ic(f'node {node} is outside')
            return True
        visited.add(node)
        if tron:
            ic(visited, frontier)
        for nb in get_empty_neighbours(_map, node):
            if nb not in visited:
                frontier.add(nb)
    return False


def is_isolated_inside(_map, coord, bounds, tron=False):
    assert is_empty(_map, coord)
    if tron:
        ic('is_isolated_inside starts', coord)
    if not find_exit(_map, coord, bounds):  # Is Internal
        return len(list(get_local_neighbours(_map, coord)))
    return 0


blocks, cube_map, bounds = load_input("cube.txt")
assert is_empty(cube_map, (2, 2, 0))
assert not is_empty(cube_map, (1, 2, 0))
assert is_isolated_inside(cube_map, (2, 2, 0), bounds) == 6

In [30]:
def solution_two(filename, tron=False):
    blocks, space_map, bounds = load_input(filename)
    acc = 0
    for coord in blocks:
        acc += count_free_sides(space_map, coord)
    (min_x, max_x), (min_y, max_y), (min_z, max_z) = bounds

    for x, y, z in itertools.product(
        range(min_x, max_x),
        range(min_y, max_y),
        range(min_z, max_z),
        ):
        if is_empty(space_map, (x, y, z)):
            inc = is_isolated_inside(space_map, (x, y, z), bounds)
            acc -= inc
    return acc

In [31]:
assert solution_two("simple.txt") == 10
assert solution_two("sample.txt") == 58

**What is the exterior surface area of your scanned lava droplet?**

In [31]:
sol = solution_two("input.txt")
print("Solution part two:", solution_two("input.txt"))

Solution part two: 2588
